<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Estimated time needed: **60** minutes.

## Introduction
Using this Python notebook you will:

1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions 


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones. 

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage. 


Therefore if we can determine if the first stage will land, we can determine the cost of a launch. 

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

 <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv" target="_blank">Spacex DataSet</a>



### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import warnings
warnings.filterwarnings('ignore')

In [ ]:
load_dotenv()

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_SERVER')
port = os.getenv('DB_PORT')
db = os.getenv('DB_NAME')

# SQLAlchemy kullanıyorsan:
database_url = f'postgresql://{user}:{password}@{host}:{port}/{db}'

In [ ]:
engine = create_engine(database_url)

In [ ]:
try:
    with engine.connect() as conn:
        result = conn.execute(text('select version()'))
        print('Bağlantı Başarılı')
        print(f'PostgreSQL versiyonu: {result.fetchone()[0]}')
except Exception as e:
    print(f'Bağlantı Hatası {e}')

In [ ]:
df = pd.read_csv('Spacex.csv')

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
df['Time (UTC)'] = pd.to_datetime(df['Time (UTC)'], format='%H:%M:%S')

In [ ]:
df['PAYLOAD_MASS__KG_'] = df['PAYLOAD_MASS__KG_'].astype('int')

In [ ]:
df['Time (UTC)'] = df['Time (UTC)'].dt.time

In [ ]:
df.to_sql('spacex', con=engine, if_exists='replace', index=False)

In [ ]:
df.info()

In [ ]:
df = df.rename(columns={'Date':'date', 'Time (UTC)':'time', 'Booster_Version':'booster_version',
'Launch_Site':'launch_site', 'Payload':'payload', 'PAYLOAD_MASS__KG_':'payload_mass_kg',
'Orbit':'orbit', 'Customer' : 'customer', 'Mission_Outcome':'mission_outcome', 
'Landing_Outcome':'landing_outcome'})

In [ ]:
df.to_sql(name='spacex', con=engine, if_exists='replace', index=False)

In [ ]:
%load_ext sql

In [ ]:
%sql postgresql://{user}:{password}@{host}:{port}/{db}

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [ ]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
%sql select distinct launch_site from spacex

In [ ]:
%config SqlMagic.autopandas = True

In [ ]:
unique_launch_site = %sql select distinct launch_site from spacex
unique_launch_site.head()


### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA' 


In [ ]:
%sql select * from spacex where launch_site like 'CCA%' limit 5

### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [ ]:
%sql select sum(payload_mass_kg) from spacex where customer like '%%NASA%%'

### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [ ]:
%%sql 

select round(avg(payload_mass_kg), 2) as avg_payload_mass 
from spacex where booster_version like 'F9 v1.1%%'

### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.


_Hint:Use min function_ 


In [ ]:
%%sql 
select date, landing_outcome from spacex 
where landing_outcome like 'Success%%' order by date limit 1

### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [ ]:
%%sql
select * from spacex 
where landing_outcome='Success (drone ship)' and payload_mass_kg between 4000 and 6000

### Task 7




##### List the total number of successful and failure mission outcomes


In [ ]:
%%sql
select mission_outcome, count(*) from spacex group by mission_outcome

In [ ]:
%%sql
    select 
    case 
        when mission_outcome like '%%Success%%' then 'success'
        else 'failure'
    end as basari,
    count(*)
    from spacex group by basari

In [ ]:
%%sql
with basari_durumu as (
    select
    case
        when mission_outcome ilike '%%success%%' then 'success'
        else 'failure'
    end as basari
    from spacex
)
select basari, count(*) from basari_durumu group by basari

### Task 8



##### List all the booster_versions that have carried the maximum payload mass, using a subquery with a suitable aggregate function.


In [ ]:
%%sql
select booster_version, payload_mass_kg from spacex
where payload_mass_kg = (select max(payload_mass_kg) from spacex)

### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [ ]:
%%sql
select to_char(date, 'Month YYYY') as ay_ismi, landing_outcome, booster_version, launch_site 
from spacex where landing_outcome ilike 'failure (drone ship)' and extract(year from date) = 2015

### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [ ]:
%%sql
select landing_outcome, count(*)
from spacex
where date between '2010-06-04' and '2017-03-20'
group by landing_outcome

# TASK 11

* Fırlatma Sırası: Her bir fırlatma sahası (Launch_Site) özelinde, fırlatmaları tarihine göre eskiden yeniye 1, 2, 3... diye numaralandır.

* Önceki Yük (Previous Payload): Her fırlatma için, aynı fırlatma sahasından yapılan bir önceki fırlatmanın yük miktarını (PAYLOAD_MASS__KG_) yan sütuna getir.

* Kümülatif Toplam: Her fırlatma sahası için, fırlatma tarihine göre o ana kadar taşınan toplam yük miktarını (koşan toplam) hesapla.

Sıralama için: ROW_NUMBER() OVER(PARTITION BY ... ORDER BY ...)

Önceki satıra erişmek için: LAG(...) OVER(PARTITION BY ... ORDER BY ...)

Kümülatif toplam için: SUM(...) OVER(PARTITION BY ... ORDER BY ...)

In [ ]:
%%sql
select booster_version,
row_number() over w as firlatma_numarasi,
lag(payload_mass_kg, 1, 0) over w as onceki_mass_kg,
sum(payload_mass_kg) over w as kumulatif_toplam
from spacex window w as (partition by launch_site order by date)

In [ ]:
%%sql
select *, row_number() over (partition by launch_site order by date) as firlatma_numarasi,
lag(payload_mass_kg, 1, 0) over (partition by launch_site order by date),
sum(payload_mass_kg) over (partition by launch_site order by date)
from spacex

### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
